In [1]:
!pip3 install beautifulsoup4 matplotlib
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import urllib
import re
from bs4 import BeautifulSoup

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
document = urllib.request.urlopen("https://www.cpubenchmark.net/multi_cpu.html").read()
soup = BeautifulSoup(document)

In [10]:
MIN_PASSMARK_SCORE = 22_000

def parse_row(row):
    # name, score, price = row.find_all("td")
    
    if row.select("span.prdname") == None:
        print(row)
    
    name = row.find("span", class_="prdname").get_text().strip()
    score = row.find("span", class_="count")
    passmark = row.find("span", class_="more_details")
    price = row.find("span", class_="price-neww").get_text().strip()
    
    if not score or score.get_text().strip() == "NA":
        score = None
    else:
        score = score.get_text().strip()
        score = float(score)
        
    if score:
        onclick = passmark['onclick']
        passmark = int(re.match("^pa?\(event, '([\d,]+)',", onclick)[1].replace(",", ""))
    else:
        passmark = None
        
    return name, score, passmark, price

In [11]:
chart = soup.select("#value .chart ul.chartlist")[0]
rows = chart.select("li")

rows = map(parse_row, rows)

# Filter out rows without a price/performance score
rows = filter(lambda row: row[1] is not None, rows)
# Sort by score
rows = sorted(rows, key=lambda row: row[1] , reverse=True)
# Filter out rows with a passmark score lower than MIN_PASSMARK_SCORE
rows = list(filter(lambda row: row[2] > MIN_PASSMARK_SCORE, rows))

In [12]:
for name, score, passmark, price in rows:
    print("{}: {} - {} ({})".format(name, score, passmark, price))

[Quad CPU] Intel Xeon E5-4627 v2 @ 3.30GHz: 78.3 - 24681 ($315.36*)
[Quad CPU] Intel Xeon E5-4650 v2 @ 2.40GHz: 68.2 - 26203 ($384.16*)
[Dual CPU] Intel Xeon E5-2678 v3 @ 2.50GHz: 67.9 - 23074 ($340.00*)
[Dual CPU] Intel Xeon E5-2670 v3 @ 2.30GHz: 67.8 - 23725 ($350.00)
[Quad CPU] Intel Xeon E7-8880 v2 @ 2.50GHz: 65.1 - 28569 ($438.56*)
[Dual CPU] Intel Xeon E5-2680 v3 @ 2.50GHz: 46.4 - 24807 ($535.16)
[Dual CPU] Intel Xeon E5-2690 v3 @ 2.60GHz: 45.5 - 26279 ($578.00)
[8-Way] Intel Xeon E7-8880 v2 @ 2.50GHz: 39.8 - 34923 ($877.12*)
[Dual CPU] Intel Xeon E5-2683 v3 @ 2.00GHz: 35.4 - 24007 ($678.00)
[Dual CPU] AMD EPYC 7272: 32.3 - 44270 ($1,369.72)
[Dual CPU] Intel Xeon E5-2660 v3 @ 2.60GHz: 29.6 - 22403 ($755.88)
[Dual CPU] Intel Xeon E5-2690 v4 @ 2.60GHz: 25.8 - 27563 ($1,069.44*)
[Dual CPU] AMD EPYC 7262: 25.6 - 34227 ($1,336.22*)
[Dual CPU] Intel Xeon E5-2683 v4 @ 2.10GHz: 24.7 - 24666 ($998.00*)
[Dual CPU] Intel Xeon E5-2690 v2 @ 3.00GHz: 24.6 - 22110 ($899.90)
[Dual CPU] Intel Xeo